In [2]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick to prevent overflow
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Global storage (simulating Redis)
storage = {}

# Dictionary to hold function mappings
function_map = {}  # { "layer_0": ["neuron_0_0", "neuron_0_1", ...], "layer_1": [...] }

# Function registry (will hold dynamically created functions)
function_registry = {}

# Function to create neuron functions dynamically
def create_neuron_function(weights, bias, activation, layer_id, neuron_id, is_final_layer):
    def neuron():
        # Determine input keys
        prev_layer = f"layer_{int(layer_id.split('_')[-1]) - 1}" if layer_id != "layer_0" else None
        input_keys = [f"input_{i}" for i in range(len(weights))] if prev_layer is None else \
                     [f"{prev_layer}_{j}" for j in range(len(weights))]

        # Fetch input values from storage
        input_values = np.array([storage.get(inp, 0) for inp in input_keys])

        if input_values.shape[0] != len(weights):
            raise ValueError(f"{layer_id}_{neuron_id} expects {len(weights)} inputs but got {input_values.shape[0]}.")

        # Compute activation
        z = np.dot(weights, input_values) + bias
        output = z if is_final_layer else ACTIVATIONS[activation](z)

        # Store output
        storage[f"{layer_id}_{neuron_id}"] = output
        return output

    function_registry[f"{layer_id}_{neuron_id}"] = neuron  # Register function

# Function to create layer functions dynamically
def create_layer_function(layer_id, neuron_ids, is_final_layer):
    def layer():
        neuron_outputs = [function_registry[neuron]() for neuron in neuron_ids]
        
        # If it's the final layer, apply softmax
        if is_final_layer:
            softmax_outputs = softmax(np.array(neuron_outputs))
            for i, val in enumerate(softmax_outputs):
                storage[f"{layer_id}_{i}"] = val

    function_registry[layer_id] = layer  # Register function

# Build network as functions
def build_network(json_data):
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neuron_ids = []

        for j, node in enumerate(layer_info['nodes']):
            neuron_id = f"{layer_name}_{j}"
            neuron_ids.append(neuron_id)

            create_neuron_function(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                layer_id=layer_name,
                neuron_id=j,
                is_final_layer=(i == len(sorted_layers) - 1)
            )

        function_map[layer_name] = neuron_ids  # Store neurons of this layer
        create_layer_function(layer_name, neuron_ids, is_final_layer=(i == len(sorted_layers) - 1))

# Forward pass using function calls
def forward_pass(layers, input_data):
    storage.clear()  # Clear storage before processing new image

    # Initialize input storage
    for i, val in enumerate(input_data):
        storage[f"input_{i}"] = val

    # Call each layer function in sequence
    for layer in layers:
        function_registry[layer]()  # Call layer function

    # Retrieve final layer outputs
    final_layer = layers[-1]
    final_outputs = np.array([storage[f"{final_layer}_{j}"] for j in range(len(function_map[final_layer]))])

    return np.argmax(final_outputs)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(list(function_map.keys()), image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")


Image 0: Label = 7, Predicted = 7
Image 1: Label = 2, Predicted = 2
Image 2: Label = 1, Predicted = 1
Image 3: Label = 0, Predicted = 0
Image 4: Label = 4, Predicted = 4
Image 5: Label = 1, Predicted = 1
Image 6: Label = 4, Predicted = 4
Image 7: Label = 9, Predicted = 9
Image 8: Label = 5, Predicted = 6
Image 9: Label = 9, Predicted = 9


In [7]:
import json
import numpy as np
from torchvision import datasets, transforms

# Simulated Redis-like storage
storage = {}

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Function to initialize a neuron dynamically
def create_neuron_function(neuron_id, weights, bias, activation, is_final_layer=False):
    activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
    
    def neuron(input_keys):
        # Fetch input values from storage
        input_values = np.array([storage.get(inp, 0) for inp in input_keys])

        # Ensure correct input size for dot product
        if input_values.shape[0] != weights.shape[0]:
            raise ValueError(f"{neuron_id} expects {weights.shape[0]} inputs but got {input_values.shape[0]}")

        # Compute neuron output
        z = np.dot(weights, input_values) + bias
        output = z if is_final_layer else activation_func(z)

        # Store neuron output in storage
        storage[neuron_id] = output
        return neuron_id  # Return the key where the output is stored

    return neuron

# Function to initialize a layer dynamically
def create_layer_function(layer_id, neurons, is_first_layer=False, input_size=None):
    """Creates a function representing a layer that triggers its neurons."""
    
    def layer(input_keys):
        output_keys = []
        for i, neuron in enumerate(neurons):
            neuron_key = f"{layer_id}_{i}"
            
            # First layer reads directly from input pixels
            neuron_inputs = input_keys if not is_first_layer else [f"input_{j}" for j in range(input_size)]

            # Call the neuron function
            output_keys.append(neuron(neuron_inputs))
        return output_keys  # Return the keys of outputs for next layer

    return layer

# Output layer function (handles softmax + final prediction)
def output_layer(last_layer_keys):
    """Final layer that computes softmax and makes the prediction."""
    # Read last hidden layer activations from storage
    inputs = np.array([storage[key] for key in last_layer_keys])

    # Apply softmax
    softmax_outputs = softmax(inputs)
    
    # Store softmax outputs
    for i, val in enumerate(softmax_outputs):
        storage[f"output_{i}"] = val

    # Final predicted class
    predicted_class = np.argmax(softmax_outputs)

    print(f"Final softmax output: {softmax_outputs}")
    print(f"Predicted class: {predicted_class}")

    return predicted_class

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Build network as functions
def build_network(json_data):
    layers = {}  # Dictionary to store layer functions
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))  # Ensure correct order

    prev_layer_size = None  # Keep track of previous layer size

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = []
        
        for j, node in enumerate(layer_info['nodes']):
            weights = np.array(node['weights'])
            biases = np.array(node['biases'])
            activation = node['activation']

            neuron_id = f"{layer_name}_{j}"
            is_final_layer = (i == len(sorted_layers) - 1)
            
            neurons.append(create_neuron_function(neuron_id, weights, biases, activation, is_final_layer))

        # Determine input size for the first layer
        if i == 0:
            input_size = len(layer_info["nodes"])  # Input size is the number of nodes in the first layer
        else:
            input_size = prev_layer_size  # Otherwise, input size is number of neurons in previous layer

        # Create layer function and store it
        layers[layer_name] = create_layer_function(layer_name, neurons, is_first_layer=(i == 0), input_size=input_size)

        # Update previous layer size for next iteration
        prev_layer_size = len(layer_info["nodes"])
    
    return layers

# Forward pass
def forward_pass(layers, input_data):
    """Simulates the forward pass using functions and storage dictionary."""
    
    # Store input in storage
    input_keys = [f"input_{i}" for i in range(len(input_data))]
    for i, val in enumerate(input_data):
        storage[input_keys[i]] = val

    # Process each layer
    current_input_keys = input_keys
    for layer_name, layer_function in layers.items():
        current_input_keys = layer_function(current_input_keys)

    # Call the output layer function
    return output_layer(current_input_keys)

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(network, image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")


ValueError: layer_0_0 expects 784 inputs but got 128